# 11 - Counterfactual Calculation

In [147]:
import pandas as pd
import pickle

In [148]:
# Display all rows and columns
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)   # Show all columns

In [149]:
df = pd.read_parquet("../data/processed/X_test.parquet")

In [150]:
df.head()

,Position_IT Support,Position_Production Technician I,Position_Area Sales Manager,Position_Production Manager,Position_Production Technician II,Position_Sales Manager,Position_Enterprise Architect,Position_Network Engineer,Position_Sr. Network Engineer,Position_Database Administrator,Position_Data Analyst,Position_Software Engineer,Position_Sr. DBA,Position_Sr. Accountant,Position_Administrative Assistant,Position_Accountant I,Position_Shared Services Manager,Position_IT Director,Position_CIO,Position_Principal Data Architect,Position_IT Manager - DB,Position_IT Manager - Support,Position_IT Manager - Infra,Position_BI Developer,Position_Senior BI Developer,Position_Data Architect,Position_BI Director,Position_Director of Sales,Position_Director of Operations,Position_Software Engineering Manager,Position_President & CEO,State,Sex,CitizenDesc_US Citizen,CitizenDesc_Eligible NonCitizen,CitizenDesc_Non-Citizen,HispanicLatino,RaceDesc_White,RaceDesc_Black or African American,RaceDesc_Asian,RaceDesc_American Indian or Alaska Native,RaceDesc_Hispanic,RaceDesc_Two or more races,Department_IT/IS,Department_Production,Department_Sales,Department_Software Engineering,Department_Admin Offices,Department_Executive Office,Age,YearsExperience,AgeGroup,ExperienceCategory,Education,Advanced Backup Strategies,Advanced Budget Forecasting,Advanced CRM Tools,Advanced Data Modeling,Advanced Data Visualization,Advanced Financial Reporting,Advanced Firewall Configurations,Advanced ITSM Tools,Advanced Machinery Maintenance,Advanced Machinery Troubleshooting,Advanced Network Configuration,Advanced Predictive Modeling,Advanced Revenue Analysis,Advanced SQL Optimization,Advanced Troubleshooting Techniques,Advanced Visualization,Agile Development Leadership,Audit Assistance,Audit Management,Backup Strategies,Backup and Recovery,Basic Accounting,Basic Machinery Maintenance,Big Data Architecture,Big Data Solutions,Budget Oversight,Budget Planning,Budget Strategy,Business Intelligence Strategy,Business Intelligence Tools,Business-IT Alignment,CI/CD Pipeline Management,Cloud Data Management,Cloud Data Solutions,Cloud Database Solutions,Cloud Integration,Cloud Networking,Cloud Strategy,Cloud-Native Data Architectures,Code Review Practices,Competitor Analysis,Cost Reduction Techniques,Customer Communication,Customer Relationship Management,Customer Retention,Customer Support,Customer Support Strategies,Cybersecurity Oversight,Dashboard Creation,Data Governance,Data Lake Architecture,Data Modeling,Data Pipeline Optimization,Data Pipeline Scalability,Data Security,Data Visualization,Database Design,Database Management,Database Tuning,Disaster Recovery Planning,Distributed Database Management,Document Management,ETL Automation,ETL Development,ETL Optimization,Efficiency Optimization,Enterprise Data Strategy,Financial Management,Financial Reporting,Firewall Expertise,Firewall Management,Forensic Accounting Techniques,Governance and Standards,Hardware Maintenance,Hardware Management,Hybrid Cloud Infrastructure Management,IT Governance,IT Security Oversight,IT Support Management,Incident Response Planning,Infrastructure Design,Java,Leadership,Leadership Skills,Lean Manufacturing,Machine Learning,Machine Learning Integration,Market Analysis,Microservices Architecture Design,Negotiation,Network Configuration,Network Management,Network Performance Optimization,Network Security Design,Office Coordination,Operations Performance Metrics,Operations Strategy,Performance Tuning,Predictive Analytics Integration,Preventive Maintenance Planning,Problem Identification,Problem-Solving,Process Improvement,Process Optimization,Production Line Efficiency Analysis,Public Relations,Python,Quality Assurance,QuickBooks,Real-Time Data Processing,Revenue Optimization,Risk Assessment,SD-WAN Deployment,SQL,SQL Optimization,Safety Protocols,Sales Funnel Optimization,Sales Strategy,Scheduling,Service Delivery Optimization,Software Design,Solution Architecture,Statistical Analysis,Strategic I

In [151]:
# Load the model
model_path = "../models/xgb_model.pkl"
with open(model_path, 'rb') as file:
    xgb_model = pickle.load(file)

In [152]:
probs = xgb_model.predict_proba(df)[:, 1]  # Get the probability of the positive class

In [153]:
# Append probabilities as a new column in the dataframe
df['probability'] = probs

In [154]:
# Define thresholds for filtering
threshold_lower = 0.40
threshold_upper = 0.60

In [155]:
# Filter rows based on probabilities being close to the threshold range (e.g., 0.45 to 0.5)
filtered_df = df[(df['probability'] >= threshold_lower) & (df['probability'] <= threshold_upper)]

In [156]:
df.YearsExperience.value_counts()

YearsExperience
13    76
10    70
11    49
12    42
14    25
9     18
8     14
16     8
7      4
15     3
17     3
19     1
Name: count, dtype: int64

In [157]:
filtered_df.Sex.value_counts()

Sex
0    6
1    4
Name: count, dtype: int64

In [158]:
row_to_modify = filtered_df.iloc[1].copy()

In [159]:
# Display the original row and its probability
print("Original Row:")
print(row_to_modify)

Original Row:
Position_IT Support                                             0.000000
Position_Production Technician I                                1.000000
Position_Area Sales Manager                                     0.000000
Position_Production Manager                                     0.000000
Position_Production Technician II                               0.000000
Position_Sales Manager                                          0.000000
Position_Enterprise Architect                                   0.000000
Position_Network Engineer                                       0.000000
Position_Sr. Network Engineer                                   0.000000
Position_Database Administrator                                 0.000000
Position_Data Analyst                                           0.000000
Position_Software Engineer                                      0.000000
Position_Sr. DBA                                                0.000000
Position_Sr. Accountant              

In [160]:
# Modify the row: change RaceDesc_White from 1 to 0 and RaceDesc_Asian from 0 to 1
if 'RaceDesc_White' in row_to_modify.index and 'RaceDesc_Asian' in row_to_modify.index:
    row_to_modify['RaceDesc_White'] = 0
    row_to_modify['RaceDesc_Asian'] = 1

In [161]:
# Display the modified row
print("\nModified Row:")
print(row_to_modify)


Modified Row:
Position_IT Support                                             0.000000
Position_Production Technician I                                1.000000
Position_Area Sales Manager                                     0.000000
Position_Production Manager                                     0.000000
Position_Production Technician II                               0.000000
Position_Sales Manager                                          0.000000
Position_Enterprise Architect                                   0.000000
Position_Network Engineer                                       0.000000
Position_Sr. Network Engineer                                   0.000000
Position_Database Administrator                                 0.000000
Position_Data Analyst                                           0.000000
Position_Software Engineer                                      0.000000
Position_Sr. DBA                                                0.000000
Position_Sr. Accountant             

In [162]:
# Convert the row back into a DataFrame (model expects a DataFrame for predictions)
modified_row_df = row_to_modify.to_frame().T
probability = modified_row_df.probability
probability

159    0.437411
Name: probability, dtype: float64

In [163]:
modified_row_df.drop(columns="probability", inplace=True)

In [164]:
# Recalculate the probability for the modified row
counterfactual_probability = xgb_model.predict_proba(modified_row_df)[:, 1][0]

# Display the counterfactual probability
print("\nCounterfactual Probability:")
print(counterfactual_probability)


Counterfactual Probability:
0.30037785


The counterfactual probability of **0.99927** shows that changing `RaceDesc_White` to 0 and `RaceDesc_Asian` to 1 caused the model to predict the positive class with almost 100% confidence. This suggests the model might be overly sensitive to these features, possibly due to their high importance or correlations in the training data.

## **Modifiable Attributes : gender, age, race**

In [165]:
def modify_and_recalculate_probability(row: pd.Series, 
                                      xgb_model: object, 
                                      attribute: str, 
                                      new_value: int | str) -> dict:
    """
    Modify a single attribute of a selected row, recalculate probability, and return results.

    Parameters:
    row (pd.Series): The original row selected by the user.
    xgb_model: The trained XGBoost model for predicting probabilities.
    attribute (str): The attribute to modify ('Sex', 'Race', or 'YearsExperience').
    new_value: The new value to set for the selected attribute.
        - For 'Sex': 0 (Female) or 1 (Male)
        - For 'Race': 'White', 'Black', or 'Asian'
        - For 'YearsExperience': 7, 13, or 17

    Returns:
    dict: A dictionary containing the original probability, modified row, and counterfactual probability.
    """
    import pandas as pd

    # Validate inputs
    if attribute not in ['Sex', 'Race', 'YearsExperience']:
        raise ValueError("Invalid attribute. Must be 'Sex', 'Race', or 'YearsExperience'.")

    if attribute == 'Sex' and new_value not in [0, 1]:
        raise ValueError("Invalid value for 'Sex'. Must be 0 (Female) or 1 (Male).")

    if attribute == 'Race' and new_value not in ['White', 'Black', 'Asian']:
        raise ValueError("Invalid value for 'Race'. Must be 'White', 'Black', or 'Asian")

    if attribute == 'YearsExperience' and new_value not in [7, 13, 17]:
        raise ValueError("Invalid value for 'YearsExperience'. Must be 7, 13, or 17.")

    # Copy the row to avoid modifying the original data
    modified_row = row.copy()

    # Save the original probability
    original_probability = modified_row.get('probability', None)
    if original_probability is None:
        raise KeyError("The column 'probability' is missing in the row.")

    # Modify the specified attribute
    if attribute == 'Sex':
        modified_row['Sex'] = new_value
    elif attribute == 'Race':
        # Dynamically identify all columns that start with 'RaceDesc_'
        race_columns = [col for col in modified_row.index if col.startswith('RaceDesc_')]
        # Reset all race columns to 0
        for col in race_columns:
            modified_row[col] = 0

        # Set the selected race column to 1
        race_column_mapping = {
            'White': 'RaceDesc_White',
            'Black': 'RaceDesc_Black or African American',
            'Asian': 'RaceDesc_Asian'
        }
        race_col = race_column_mapping.get(new_value)
        if race_col and race_col in modified_row:
            modified_row[race_col] = 1
    elif attribute == 'YearsExperience':
        modified_row['YearsExperience'] = new_value

    # Convert the modified row to a DataFrame (XGBoost model expects this format)
    modified_row_df = modified_row.to_frame().T
    modified_row_df.drop(columns="probability", inplace=True)

    # Recalculate probability for the modified row
    counterfactual_probability = xgb_model.predict_proba(modified_row_df)[:, 1][0]

    return {
        "original_probability": original_probability,
        "counterfactual_probability": counterfactual_probability,
        "modified_row": modified_row
    }

In [166]:
result = modify_and_recalculate_probability(row_to_modify, xgb_model, 'Race', 'Asian')
print(result)

{'original_probability': np.float64(0.4374110996723175), 'counterfactual_probability': np.float32(0.30037785), 'modified_row': Position_IT Support                                             0.000000
Position_Production Technician I                                1.000000
Position_Area Sales Manager                                     0.000000
Position_Production Manager                                     0.000000
Position_Production Technician II                               0.000000
Position_Sales Manager                                          0.000000
Position_Enterprise Architect                                   0.000000
Position_Network Engineer                                       0.000000
Position_Sr. Network Engineer                                   0.000000
Position_Database Administrator                                 0.000000
Position_Data Analyst                                           0.000000
Position_Software Engineer                                      0.0000

In [167]:
# Filter for a row where 'RaceDesc_White' is 1
white_row = df[df['RaceDesc_White'] == 1].iloc[0]

# Change race from 'White' to 'Black'
result = modify_and_recalculate_probability(white_row, xgb_model, 'Race', 'Black')

print("Original Probability:", result['original_probability'])
print("Modified Row:")
print(result['modified_row'])
print("Counterfactual Probability:", result['counterfactual_probability'])

Original Probability: 0.7853830456733704
Modified Row:
Position_IT Support                                             0.000000
Position_Production Technician I                                0.000000
Position_Area Sales Manager                                     0.000000
Position_Production Manager                                     0.000000
Position_Production Technician II                               1.000000
Position_Sales Manager                                          0.000000
Position_Enterprise Architect                                   0.000000
Position_Network Engineer                                       0.000000
Position_Sr. Network Engineer                                   0.000000
Position_Database Administrator                                 0.000000
Position_Data Analyst                                           0.000000
Position_Software Engineer                                      0.000000
Position_Sr. DBA                                                0.000

In [168]:
# Filter for a row where 'Sex' is 1 (Male)
male_row = filtered_df[filtered_df['Sex'] == 1].iloc[0]

# Change gender from Male to Female
result = modify_and_recalculate_probability(male_row, xgb_model, 'Sex', 0)

print("Original Probability:", result['original_probability'])
print("Modified Row:")
print(result['modified_row'])
print("Counterfactual Probability:", result['counterfactual_probability'])

Original Probability: 0.4374110996723175
Modified Row:
Position_IT Support                                             0.000000
Position_Production Technician I                                1.000000
Position_Area Sales Manager                                     0.000000
Position_Production Manager                                     0.000000
Position_Production Technician II                               0.000000
Position_Sales Manager                                          0.000000
Position_Enterprise Architect                                   0.000000
Position_Network Engineer                                       0.000000
Position_Sr. Network Engineer                                   0.000000
Position_Database Administrator                                 0.000000
Position_Data Analyst                                           0.000000
Position_Software Engineer                                      0.000000
Position_Sr. DBA                                                0.000

In [169]:
# Filter for a row with a specific value for 'YearsExperience' (e.g., 7, 13, or 17)
years_row = df[df['YearsExperience'] == 7].iloc[1]

# Change 'YearsExperience' to a new value (e.g., 13)
result = modify_and_recalculate_probability(years_row, xgb_model, 'YearsExperience', 17)

print("Original Probability:", result['original_probability'])
print("Modified Row:")
print(result['modified_row'])
print("Counterfactual Probability:", result['counterfactual_probability'])


Original Probability: 0.9999783039093018
Modified Row:
Position_IT Support                                             0.000000
Position_Production Technician I                                0.000000
Position_Area Sales Manager                                     0.000000
Position_Production Manager                                     0.000000
Position_Production Technician II                               0.000000
Position_Sales Manager                                          0.000000
Position_Enterprise Architect                                   0.000000
Position_Network Engineer                                       0.000000
Position_Sr. Network Engineer                                   0.000000
Position_Database Administrator                                 0.000000
Position_Data Analyst                                           0.000000
Position_Software Engineer                                      0.000000
Position_Sr. DBA                                                0.000

In [170]:
# Filter for a row with a specific value for 'YearsExperience' (e.g., 7, 13, or 17)
years_row = filtered_df[filtered_df['YearsExperience'] == 13].iloc[0]

# Change 'YearsExperience' to a new value (e.g., 17)
result = modify_and_recalculate_probability(years_row, xgb_model, 'YearsExperience', 17)

print("Original Probability:", result['original_probability'])
print("Modified Row:")
print(result['modified_row'])
print("Counterfactual Probability:", result['counterfactual_probability'])

Original Probability: 0.4866895377635956
Modified Row:
Position_IT Support                                             0.00000
Position_Production Technician I                                1.00000
Position_Area Sales Manager                                     0.00000
Position_Production Manager                                     0.00000
Position_Production Technician II                               0.00000
Position_Sales Manager                                          0.00000
Position_Enterprise Architect                                   0.00000
Position_Network Engineer                                       0.00000
Position_Sr. Network Engineer                                   0.00000
Position_Database Administrator                                 0.00000
Position_Data Analyst                                           0.00000
Position_Software Engineer                                      0.00000
Position_Sr. DBA                                                0.00000
Position_

By changing individual attributes like race, gender, or years of experience, we observed significant shifts in the model's predictions. These results highlight the sensitivity of the model to specific features and emphasize the need to evaluate feature importance and potential biases. This demonstrates the impact of single-value modifications on prediction outcomes.

In [171]:
# TODO: Modify this with Age

In [172]:
# Classification threshold (e.g., 0.5)
classification_threshold = 0.5

candidates = []

# Define attributes and values to modify
attributes = [
    {"attribute": "Sex", "values": [0, 1]},  # Female to Male or vice versa
    {"attribute": "Race", "values": ["White", "Black", "Asian"]},  # Race changes (mapped to columns)
    {"attribute": "YearsExperience", "values": [7, 13, 17]}  # Experience levels
]

# Mapping for race columns (one-hot encoded)
race_column_mapping = {
    "White": "RaceDesc_White",
    "Black": "RaceDesc_Black or African American",
    "Asian": "RaceDesc_Asian"
}

# Mapping for ExperienceCategory based on YearsExperience
experience_category_mapping = {
    7: 0,   # 6-10 years -> ExperienceCategory = 0
    13: 1,  # 11-20 years -> ExperienceCategory = 1
    17: 1   # 11-20 years -> ExperienceCategory = 1
}

# Iterate through rows to identify significant changes
for idx, row in df.iterrows():
    for attr in attributes:
        if attr["attribute"] == "Race":
            # Handle one-hot encoding for race
            for race, race_col in race_column_mapping.items():
                if race_col in row and row[race_col] == 1:  # Skip if current race matches target race
                    continue

                # Create a modified copy of the row for race modification
                modified_row = row.copy()

                # Reset all race columns to 0
                for col in race_column_mapping.values():
                    if col in modified_row:
                        modified_row[col] = 0

                # Set the target race column to 1
                if race_col in modified_row:
                    modified_row[race_col] = 1

                # Apply modification and calculate probabilities
                result = modify_and_recalculate_probability(modified_row, xgb_model, "Race", race)
                original_prob = result["original_probability"]
                counterfactual_prob = result["counterfactual_probability"]

                # Identify the original race
                original_race = next(
                    (k for k, v in race_column_mapping.items() if v in row and row[v] == 1),
                    "Unknown"
                )

                # Check if the change crosses the classification threshold
                if (original_prob < classification_threshold <= counterfactual_prob) or \
                   (original_prob >= classification_threshold > counterfactual_prob):
                    candidates.append({
                        "row_index": idx,
                        "attribute": "Race",
                        "original_value": original_race,
                        "new_value": race,
                        "original_probability": original_prob,
                        "counterfactual_probability": counterfactual_prob,
                        "probability_change": abs(counterfactual_prob - original_prob)
                    })
        elif attr["attribute"] == "YearsExperience":
            # Handle YearsExperience changes
            for value in attr["values"]:
                if row[attr["attribute"]] == value:  # Skip if the current value matches the new value
                    continue

                # Create a modified copy of the row
                modified_row = row.copy()

                # Set the new value for YearsExperience
                modified_row[attr["attribute"]] = value

                # Set the corresponding ExperienceCategory based on the mapping
                if value in experience_category_mapping:
                    modified_row["ExperienceCategory"] = experience_category_mapping[value]

                # Apply modification and calculate probabilities
                result = modify_and_recalculate_probability(modified_row, xgb_model, attr["attribute"], value)
                original_prob = result["original_probability"]
                counterfactual_prob = result["counterfactual_probability"]

                # Check if the change crosses the classification threshold
                if (original_prob < classification_threshold <= counterfactual_prob) or \
                   (original_prob >= classification_threshold > counterfactual_prob):
                    candidates.append({
                        "row_index": idx,
                        "attribute": attr["attribute"],
                        "original_value": row[attr["attribute"]],
                        "new_value": value,
                        "original_probability": original_prob,
                        "counterfactual_probability": counterfactual_prob,
                        "probability_change": abs(counterfactual_prob - original_prob),
                    })
        else: 
            # Handle other attributes (e.g., Sex)
            for value in attr["values"]:
                if row[attr["attribute"]] == value:  # Skip if the current value matches the new value
                    continue

                # Apply modification and calculate probabilities
                result = modify_and_recalculate_probability(row, xgb_model, attr["attribute"], value)
                original_prob = result["original_probability"]
                counterfactual_prob = result["counterfactual_probability"]

                # Check if the change crosses the classification threshold
                if (original_prob < classification_threshold <= counterfactual_prob) or \
                   (original_prob >= classification_threshold > counterfactual_prob):
                    candidates.append({
                        "row_index": idx,
                        "attribute": attr["attribute"],
                        "original_value": row[attr["attribute"]],
                        "new_value": value,
                        "original_probability": original_prob,
                        "counterfactual_probability": counterfactual_prob,
                        "probability_change": abs(counterfactual_prob - original_prob)
                    })

In [173]:
# Sort by probability change for better insights
candidates_df = pd.DataFrame(candidates)
candidates_df = candidates_df.sort_values(by="probability_change", ascending=False)
candidates_df = candidates_df.reset_index(drop=True)

In [174]:
# Define custom sort order for the attribute column
attribute_order = {"Sex": 0, "Race": 1, "YearsExperience": 2}

# Sort DataFrame
candidates_df = candidates_df.sort_values(
    by=["attribute", "original_probability", "probability_change"],
    key=lambda col: col.map(attribute_order) if col.name == "attribute" else col,
    ascending=[True, True, False]  # Ascending for attribute and original_probability, descending for probability_change
)
candidates_df.reset_index(drop=True, inplace=True)
candidates_df

,row_index,attribute,original_value,new_value,original_probability,counterfactual_probability,probability_change
0,104,Sex,0.0,1,0.001592,0.900683,0.899090
1,3013,Sex,0.0,1,0.005931,0.998006,0.992076
2,1417,Sex,0.0,1,0.014034,0.916710,0.902676
3,2640,Sex,0.0,1,0.028825,0.922619,0.893794
4,3041,Sex,0.0,1,0.044305,0.757632,0.713327
5,531,Sex,0.0,1,0.045195,0.710245,0.665049
6,841,Sex,0.0,1,0.055539,0.585676,0.530137
7,1466,Sex,1.0,0,0.080916,0.756877,0.675961
8,1748,Sex,1.0,0,0.136697,0.609562,0.472865
9,1533,Sex,1.0,0,0.138365,0.871441,0.733076


In [175]:
# Group by 'row_index' and count occurrences
duplicate_candidates = candidates_df.groupby("row_index").size()

# Filter for row indices that appear more than once
duplicate_row_indices = duplicate_candidates[duplicate_candidates > 1].index

# Filter original DataFrame for these row indices and select required columns
duplicates_df = candidates_df[candidates_df["row_index"].isin(duplicate_row_indices)][
    ["row_index", "attribute", "original_value", "new_value", "original_probability", "counterfactual_probability", "probability_change"]
]
# Filter out rows where 'attribute' is "Race" and 'original_value' is "Unknown"
filtered_duplicates_df = duplicates_df[
    ~((duplicates_df["attribute"] == "Race") & (duplicates_df["original_value"] == "Unknown"))
]
duplicates_df.reset_index(drop=True, inplace=True)
duplicates_df.sort_values(by="row_index")

,row_index,attribute,original_value,new_value,original_probability,counterfactual_probability,probability_change
0,104,Sex,0.0,1,0.001592,0.900683,0.899090
97,104,YearsExperience,11.0,17,0.001592,0.999925,0.998333
98,104,YearsExperience,11.0,13,0.001592,0.937597,0.936004
160,159,YearsExperience,12.0,17,0.437411,0.965634,0.528223
161,159,YearsExperience,12.0,13,0.437411,0.944531,0.507120
15,180,Sex,0.0,1,0.310481,0.818533,0.508052
152,180,YearsExperience,11.0,7,0.310481,0.965158,0.654677
153,180,YearsExperience,11.0,13,0.310481,0.712940,0.402458
151,180,YearsExperience,11.0,17,0.310481,0.999158,0.688677
75,180,Race,Unknown,Black,0.310481,0.590780,0.280299


**Note:** We can use this table to identify some interesting candidates for the MVP, as it highlights rows where multiple attributes significantly impact the classification threshold.

In [176]:
candidates_df["row_index"].value_counts()

row_index
180     7
1833    6
2553    6
841     6
1320    5
1818    5
1621    5
2619    5
2788    5
3124    5
1429    5
3013    5
1417    5
427     4
870     4
2310    4
1497    4
3041    4
2640    4
238     4
2076    4
2574    3
104     3
2024    3
1471    3
1726    3
2701    3
1540    3
1852    3
1903    3
531     3
1466    3
1533    3
642     3
527     3
1547    2
159     2
1433    2
581     2
974     2
928     2
2583    2
321     2
1748    2
1153    2
2046    2
874     2
1521    2
1743    2
494     2
868     2
2300    2
938     1
352     1
899     1
2590    1
75      1
2720    1
2065    1
56      1
2650    1
937     1
1131    1
1399    1
1821    1
332     1
289     1
2800    1
1982    1
575     1
2616    1
2274    1
2707    1
2096    1
572     1
973     1
99      1
1164    1
277     1
2882    1
2906    1
1786    1
548     1
1705    1
2825    1
2987    1
338     1
2286    1
1924    1
115     1
103     1
1136    1
1813    1
1951    1
2931    1
1484    1
2293    1
2618    1
138     1
